In [1]:
import galsim
import numpy as np

import ngmix
from ngmix.gaussmom import GaussMom
from ngmix.moments import fwhm_to_T

from deep_field_metadetect.gaussmom.gaussmom_core import obs_to_gaussmom_obs

from deep_field_metadetect.gaussmom.gaussmom import eval_gaussian_moments

In [2]:
def test_gaussmom_smoke(g1_true, g2_true, wcs_g1, wcs_g2, weight_fac):
    """ """

    fwhm = 0.9
    image_size = 107
    cen = (image_size - 1) / 2
    gs_wcs = galsim.ShearWCS(0.125, galsim.Shear(g1=wcs_g1, g2=wcs_g2)).jacobian()

    obj = galsim.Gaussian(fwhm=fwhm).shear(g1=g1_true, g2=g2_true).withFlux(400)
    im = obj.drawImage(
        nx=image_size, ny=image_size, wcs=gs_wcs, method="no_pixel"
    ).array
    # noise = np.sqrt(np.sum(im**2)) / 1e18
    # wgt = np.ones_like(im) / noise**2
    # scale = np.sqrt(gs_wcs.pixelArea())

    # get true flux
    ngmix_jac = ngmix.Jacobian(
        y=cen,
        x=cen,
        dudx=gs_wcs.dudx,
        dudy=gs_wcs.dudy,
        dvdx=gs_wcs.dvdx,
        dvdy=gs_wcs.dvdy,
    )
    ngmix_obs = ngmix.Observation(
        image=im,
        jacobian=ngmix_jac,
    )

    gaussmom_obs = obs_to_gaussmom_obs(obs=ngmix_obs)

    T = fwhm_to_T(fwhm=fwhm)
    sigma = np.sqrt(T / 2)
    maxrad = 100 * sigma

    res_jax = eval_gaussian_moments(
        gaussmom_obs=gaussmom_obs, fwhm=fwhm * weight_fac, maxrad=maxrad
    )

    # run ngmix
    fitter = GaussMom(fwhm=fwhm * weight_fac)
    # get true flux
    res_ngmix = fitter.go(obs=ngmix_obs)

    return res_jax, res_ngmix

In [3]:
jax_res, ngmix_res = test_gaussmom_smoke(0.1, -0.2, -0.5, 0.2, 1e5)

In [4]:
for key, value in jax_res._asdict().items():
    print(f"{key}:{value}")

obs:GaussMomObs(u=Array([[-10.22115703,  -9.99863547,  -9.77611391, ...,  12.9210853 ,
         13.14360686,  13.36612843],
       [-10.25082657, -10.02830501,  -9.80578345, ...,  12.89141576,
         13.11393732,  13.33645888],
       [-10.28049611, -10.05797455,  -9.83545299, ...,  12.86174622,
         13.08426778,  13.30678934],
       ...,
       [-13.30678934, -13.08426778, -12.86174622, ...,   9.83545299,
         10.05797455,  10.28049611],
       [-13.33645888, -13.11393732, -12.89141576, ...,   9.80578345,
         10.02830501,  10.25082657],
       [-13.36612843, -13.14360686, -12.9210853 , ...,   9.77611391,
          9.99863547,  10.22115703]], dtype=float64), v=Array([[-2.35872855, -2.38839809, -2.41806763, ..., -5.44436086,
        -5.4740304 , -5.50369994],
       [-2.28455469, -2.31422423, -2.34389377, ..., -5.370187  ,
        -5.39985654, -5.42952609],
       [-2.21038084, -2.24005038, -2.26971992, ..., -5.29601315,
        -5.32568269, -5.35535223],
       ...,
   

In [5]:
ngmix_res

{'flags': 0,
 'npix': 11449,
 'wsum': 11448.999787725585,
 'sums': array([ 9.50760487e-17, -7.51757417e-16,  2.46017464e+01, -4.92034941e+01,
         1.29159172e+02,  4.00000000e+02]),
 'sums_cov': array([[ 6.97068701e+04, -9.61474042e+04, -4.60431693e-12,
          2.07478479e-11, -3.03543857e-11, -8.97504293e-13],
        [-9.61474042e+04,  5.50443897e+05, -1.65982783e-11,
         -2.16232365e-10,  1.07547748e-10,  9.80548975e-13],
        [-3.36513040e-11, -3.80850906e-11,  4.08841055e+07,
         -1.64460937e+07,  4.78352379e+07,  4.80737027e+05],
        [ 2.00657269e-11,  5.62749847e-12, -1.64460937e+07,
          1.74421928e+07, -2.25743692e+07, -1.92294808e+05],
        [-4.51905180e-11,  5.32054401e-11,  4.78352379e+07,
         -2.25743692e+07,  5.83262983e+07,  6.20150767e+05],
        [-8.97504293e-13,  9.80548975e-13,  4.80737027e+05,
         -1.92294808e+05,  6.20150767e+05,  1.14489996e+04]]),
 'pars': array([ 1.48556326e-18, -1.17462096e-17,  1.90476186e-01, -3.8095